In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import glob
from model import MultiTaskUNet  # Import model mới

# --- 1. DATASET THÔNG MINH (Oversampling) ---
class SmartUnifiedDataset(Dataset):
    def __init__(self, chiu_dir, srinivasan_dir, img_size=256, oversample_factor=5):
        self.img_size = img_size
        self.samples = []
        self.class_map = {'AMD': 0, 'DME': 1, 'NORMAL': 2}
        
        # Load CHIU (Nhân bản 5 lần để cân bằng)
        if os.path.exists(chiu_dir):
            chiu_img_dir = os.path.join(chiu_dir, 'images')
            chiu_mask_dir = os.path.join(chiu_dir, 'masks')
            chiu_files = sorted(os.listdir(chiu_img_dir))
            for _ in range(oversample_factor):
                for f in chiu_files:
                    self.samples.append({
                        'img_path': os.path.join(chiu_img_dir, f),
                        'mask_path': os.path.join(chiu_mask_dir, f),
                        'label': 1, 'source': 'chiu'
                    })
            print(f"--> Chiu (x{oversample_factor}): {len(chiu_files)*oversample_factor} ảnh.")
            
        # Load DUKE
        if os.path.exists(srinivasan_dir):
            count_duke = 0
            for cls_name, label_idx in self.class_map.items():
                cls_folder = os.path.join(srinivasan_dir, cls_name)
                if not os.path.exists(cls_folder): continue
                files = glob.glob(os.path.join(cls_folder, "*.png"))
                for f_path in files:
                    self.samples.append({
                        'img_path': f_path,
                        'mask_path': None,
                        'label': label_idx, 'source': 'duke'
                    })
                    count_duke += 1
            print(f"--> Duke: {count_duke} ảnh.")

    def __len__(self): return len(self.samples)
    
    def __getitem__(self, idx):
        item = self.samples[idx]
        img = cv2.imread(item['img_path'], cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (self.img_size, self.img_size)) / 255.0
        # EfficientNet cần 3 kênh màu, ta lặp lại ảnh xám 3 lần
        img = np.stack([img, img, img], axis=0) 
        
        if item['mask_path'] is not None:
            mask = cv2.imread(item['mask_path'], cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, (self.img_size, self.img_size), interpolation=cv2.INTER_NEAREST)
            mask = (mask > 0).astype(np.float32)
            has_mask = 1.0
        else:
            mask = np.zeros((self.img_size, self.img_size), dtype=np.float32)
            has_mask = 0.0
            
        return torch.tensor(img, dtype=torch.float32), torch.tensor(mask).unsqueeze(0), torch.tensor(item['label'], dtype=torch.long), torch.tensor(has_mask, dtype=torch.float32)

# --- 2. CẤU HÌNH ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 8    # Giảm xuống 8 vì EfficientNet nặng hơn
EPOCHS = 50       # 50 Epochs với dữ liệu nhân bản là rất nhiều
LR = 1e-4

CHIU_DIR = r"D:\project\processed_data"
SRINIVASAN_DIR = r"D:\project\processed_srinivasan"

# Load Data
print("⏳ Đang nạp dữ liệu (Chế độ khô máu)...")
full_dataset = SmartUnifiedDataset(chiu_dir=CHIU_DIR, srinivasan_dir=SRINIVASAN_DIR, img_size=256, oversample_factor=5)
train_loader = DataLoader(full_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Khởi tạo Model EfficientNet (3 kênh input)
model = MultiTaskUNet(n_channels=3, n_classes_seg=1, n_classes_cls=3).to(DEVICE)
optimizer = optim.AdamW(model.parameters(), lr=LR)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

# Loss
class DiceLoss(nn.Module):
    def __init__(self, smooth=1.0):
        super(DiceLoss, self).__init__()
        self.smooth = smooth
    def forward(self, pred, target):
        pred = torch.sigmoid(pred).view(-1)
        target = target.view(-1)
        intersection = (pred * target).sum()
        dice = (2. * intersection + self.smooth) / (pred.sum() + target.sum() + self.smooth)
        return 1 - dice

criterion_cls = nn.CrossEntropyLoss()
criterion_dice = DiceLoss()
criterion_bce = nn.BCEWithLogitsLoss()

# --- 3. TRAINING ---
print(f"\n🚀 START TRAINING EFFICIENTNET-B3 ...")
model.train()

for epoch in range(EPOCHS):
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")
    epoch_loss = 0
    
    for batch_idx, (images, masks, labels, has_masks) in enumerate(loop):
        images, masks, labels, has_masks = images.to(DEVICE), masks.to(DEVICE), labels.to(DEVICE), has_masks.to(DEVICE)
        
        pred_seg, pred_cls = model(images)
        loss_cls = criterion_cls(pred_cls, labels)
        
        mask_indices = torch.nonzero(has_masks > 0).squeeze()
        if mask_indices.numel() > 0:
            if mask_indices.ndim == 0: mask_indices = mask_indices.unsqueeze(0)
            l_dice = criterion_dice(pred_seg[mask_indices], masks[mask_indices])
            l_bce = criterion_bce(pred_seg[mask_indices], masks[mask_indices])
            loss_seg = 0.7 * l_dice + 0.3 * l_bce 
        else:
            loss_seg = torch.tensor(0.0).to(DEVICE)

        # Trọng số cực mạnh
        loss = 50.0 * loss_seg + 1.0 * loss_cls
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        loop.set_postfix(loss=loss.item(), lr=optimizer.param_groups[0]['lr'])
        
    scheduler.step(epoch_loss / len(train_loader))
    
    if (epoch + 1) % 5 == 0:
        torch.save(model.state_dict(), f"effnet_epoch_{epoch+1}.pth")

print("✅ Đã xong! Kiểm tra kết quả...")

# --- 4. TÍNH DICE ---
model.eval()
dice_score = 0
num_batches = 0
with torch.no_grad():
    for x, y, labels, has_masks in train_loader:
        x, y, has_masks = x.to(DEVICE), y.to(DEVICE), has_masks.to(DEVICE)
        valid_idx = torch.nonzero(has_masks > 0).squeeze()
        if valid_idx.numel() > 0:
            if valid_idx.ndim == 0: valid_idx = valid_idx.unsqueeze(0)
            pred = (torch.sigmoid(model(x[valid_idx])[0]) > 0.5).float()
            target = y[valid_idx]
            inter = (pred * target).sum()
            union = pred.sum() + target.sum()
            dice_score += ((2. * inter + 1e-6) / (union + 1e-6)).item()
            num_batches += 1

print(f"\n🏆 FINAL DICE (EFFICIENTNET): {(dice_score/num_batches):.4f}")

⏳ Đang nạp dữ liệu gốc...
--> Đã nạp 610 ảnh từ bộ Chiu (Có Mask).
--> Đã nạp 3231 ảnh từ bộ Srinivasan (Duke).
✅ Tổng cộng: 3841 ảnh.

🚀 Bắt đầu Training Ổn định (100 Epochs)...


Epoch 100/100: 100%|██████████| 241/241 [00:50<00:00,  4.80it/s, cls=0.62, seg=0]           


✅ Training hoàn tất!

📊 Đang tính toán kết quả...
🏆 FINAL DICE SCORE: 0.6889
